In [ ]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt

In [ ]:
# initialize data frames
df_day_ahead_energy_price = pd.read_csv('day_ahead_energy_price.csv')
df_generation_by_source = pd.read_csv('generation_by_source.csv')
df_historical_power_load = pd.read_csv('historical_power_load.csv')

In [ ]:
# head of day ahead energy price
df_day_ahead_energy_price.head()

In [ ]:
# head of generation by source
df_generation_by_source.head()

In [ ]:
# head of historical power load
df_historical_power_load.head()